In [1]:
!git clone https://github.com/kuandeng/LightGCN

Cloning into 'LightGCN'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 101 (delta 3), reused 4 (delta 0), pack-reused 92
Receiving objects: 100% (101/101), 16.79 MiB | 6.90 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [2]:
%cd LightGCN
!ls

/kaggle/working/LightGCN
Data  LightGCN.py  README.md  evaluator  setup.py  utility


In [3]:
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
def load_embeddings(embeddings_file):
    # load embeddings from word2vec format file
    model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)
    features_matrix = np.asarray([model[str(node)] for node in range(len(model.index_to_key))])
    return features_matrix

In [4]:
import pandas as pd
from numpy import dot
import random
import os

import json
from tqdm.notebook import tqdm

In [5]:
path_input = {}
path_input_label = {}
path_input_past = {}
path_output = {}
for i in ["train", "valid"]:
    path_input[i] = f"/kaggle/input/otto-make-data-train-test-split/{i}.jsonl"
    path_input_label[i] = f"/kaggle/input/otto-make-data-label/label_{i}.parquet"
    path_input_past[i] = f"/kaggle/input/otto-make-data-past-event/past_event_{i}.pickle"
    os.makedirs(f"Data/{i}", exist_ok = True)
    path_output[i] = {}
    for j in ["train", "test"]:
        path_output[i][j] = f"Data/{i}/{j}.txt"
    
sample_size = 150000
N = 10

event_type = "orders"

In [6]:
# for i in ["train", "valid"]:
for i in ["valid"]:
    file = {}
    # list_aidのorder履歴のあるsessionを記録する
    list_session = []
    set_aid = set()
    chunks = pd.read_json(path_input[i], lines = True, chunksize = sample_size)
    for df_chunk in tqdm(chunks):
        for j, session in df_chunk.iterrows():
            session = session.to_dict()
            list_event = session["events"][:session["cutoff"]]
            list_aid_ses = [event["aid"] for event in list_event if event["type"] == event_type]
            if len(list_aid_ses) > 0: # 1つ以上orderがあるsessionのみ保持する
                list_session.append(session["session"])
                set_aid |= set(list_aid_ses)
    list_aid = list(set_aid)
    set_session = set(list_session)
    print(i, len(list_aid))
    print(i, len(list_session))
    
    # idとindexの変換dictを作成する
    session2idx = dict(zip(list_session, range(len(list_session))))
    aid2idx = dict(zip(list_aid, range(len(list_session),len(list_session) + len(list_aid))))

    for j in ["train", "test"]: file[j] = open(path_output[i][j], "w")
    chunks = pd.read_json(path_input[i],lines = True, chunksize = sample_size)
    for df_chunk in tqdm(chunks):
        for j, session in df_chunk.iterrows():
            if not session["session"] in set_session: continue
            list_event = session["events"][:session["cutoff"]] # validはcutoff以前のeventのみ使える
            list_aid_ses = []
            for event in list_event:
                if event["type"] == event_type and event["aid"] in set_aid:
                    list_aid_ses.append(str(event["aid"]))
            if len(list_aid_ses) > 1:
                list_aid_ses = random.sample(list_aid_ses, len(list_aid_ses))
            else:
                list_aid_ses = list_aid_ses * 2 # train_test_splitできるように要素をダブらせる
            
            file["train"].write(str(session2idx[session["session"]]) + " " + " ".join(list_aid_ses[:(len(list_aid_ses)//2)]) + "\n")
            file["test"].write(str(session2idx[session["session"]]) + " " + " ".join(list_aid_ses[(len(list_aid_ses)//2):]) + "\n")
    for k in ["train", "test"]:file[k].close()
#     emb = load_embeddings("emb_spectral.emb")
#     display(emb.shape)
#     pred = {}
#     dict_past = pd.read_pickle(path_input_past[i])
#     chunksize = 1000
#     ses_idx_from = 0
#     pred = {}
#     pbar = tqdm(total = len(list_session)//chunksize + 1)
#     while(ses_idx_from < len(list_session)):
#         if (ses_idx_from + chunksize) < len(list_session): ses_idx_to = ses_idx_from + chunksize
#         else: ses_idx_to = len(list_session)
#         list_session_chunk = list_session[ses_idx_from:ses_idx_to]   
#         list_session_idx = [session2idx[session] for session in list_session_chunk]
#         mat_dist = dot(emb[list_session_idx], emb[range(len(list_session),len(list_session) + len(list_aid))].T)
#         mat_dist_p = [[dist if list_aid[idx] not in dict_past[event_type][list_session[ses_idx_from+sesidx]] else -1 for idx, dist in enumerate(ses) ] for sesidx, ses in enumerate(mat_dist)]   
#         pred_chunk = dict(zip(list_session_chunk, [[list_aid[aididx] for aididx in np.argsort(x)[::-1][:N]] for x in mat_dist_p]))
#         pred.update(pred_chunk)
#         ses_idx_from = ses_idx_to
#         pbar.update(1)
# #         break
# #     break
#     # カバー率を計算
#     df_label = pd.read_parquet(path_input_label[i])
#     n_all, n_hit= 0,0
#     for ses, label in tqdm(zip(df_label["session"],df_label[f"label_{event_type}"])):
#         n_all += len(label)
#         pred_ses = pred.get(ses, [])
#         n_hit += sum([1 for x in pred_ses if x in label])

#     print(f"{i}: {n_hit/n_all}")


0it [00:00, ?it/s]

valid 49059
valid 40387


0it [00:00, ?it/s]

In [7]:
!python LightGCN.py --dataset valid --regs [1e-4] --embed_size 64 --layer_size [64,64,64] --lr 0.001 --batch_size 8192 --epoch 1000


eval_score_matrix_foldout with python
n_users=40387, n_items=1855595
n_interactions=103932
n_train=49623, n_test=54309, sparsity=0.00000
already create adjacency matrix (1895982, 1895982) 4083.2692527770996
generate single-normalized adjacency matrix.
/kaggle/working/LightGCN/utility/load_data.py:126: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1).flatten()
generate single-normalized adjacency matrix.
already normalize adjacency matrix 0.49552106857299805
/kaggle/working/LightGCN/utility/load_data.py:96: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -0.5).flatten()
generate pre adjacency matrix.
use the pre adjcency matrix
Traceback (most recent call last):
  File "LightGCN.py", line 462, in <module>
    model = LightGCN(data_config=config, pretrain_data=pretrain_data)
  File "LightGCN.py", line 51, in __init__
    self.users = tf.placeholder(tf.int32, shape=(None,))
AttributeError: module 'tensorflow' has no attribut

In [8]:
!head Data/valid/train.txt

0 990658 1033148 1199737
1 1366413
2 1711586
3 52798
4 1849620
5 1724341 893699
6 1812034
7 1072244 1741607
8 291698
9 1498975


In [9]:
!ls

Data  LightGCN.py  README.md  evaluator  setup.py  utility
